In [1]:
import pandas as pd
df=pd.read_json('/kaggle/input/career-json/Career_dataset_1.7.json')
df.head()

,question,answer
0,I am A.Uha Priya My gender is Female I have do...,With your educational background in Computer A...
1,I am Aadil My gender is Male I have done B.E M...,Based on your background in Computer Science E...
2,I am Aakriti My gender is Female I have done B...,"Aakriti, with your background in psychology, s..."
3,I am Aanchal sharma My gender is Female I have...,With an MBA in Commerce and a keen interest in...
4,I am Aangkeeta Sarkar My gender is Female I ha...,Based on your background in B.Tech with a majo...


In [2]:
df['question'][0]

"I am A.Uha Priya My gender is Female I have done B.Sc My Major subject is Computer Applications I have interest in Cloud computing with skillset of Python, SQL, Java and I don't have certification, with work experience in Linux, Git and I don't have job and my masters degree is Software"

## Loading tokenizer and model

In [3]:
from transformers import BartTokenizer

# Load the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['question'], padding="longest", truncation=True)

tokenized_dataset = df.apply(tokenize_function, axis=1)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq

# Load the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        inputs = tokenizer(item['question'], padding="longest", truncation=True, return_tensors="pt")
        labels = tokenizer(item['answer'], padding="longest", truncation=True, return_tensors="pt").input_ids
        return {
            'input_ids': inputs.input_ids.flatten(), 
            'attention_mask': inputs.attention_mask.flatten(), 
            'labels': labels.flatten()
        }

# Load dataset
dataset = CustomDataset(df)

# Use DataCollator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [5]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
)

# Initialize the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Initialize Trainer
trainer = Trainer(
    model=model ,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

# Fine-tune the model
trainer.train()


2024-06-04 19:34:52.026916: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 19:34:52.027008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 19:34:52.157189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wa

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,10.143000
20,9.441600
30,6.311300
40,3.996600
50,3.138100
60,2.710400
70,2.403900
80,2.214800
90,2.148200
100,2.073300


TrainOutput(global_step=447, training_loss=2.2114916295813236, metrics={'train_runtime': 411.428, 'train_samples_per_second': 8.714, 'train_steps_per_second': 1.086, 'total_flos': 883082427482112.0, 'train_loss': 2.2114916295813236, 'epoch': 2.99})

## saving the model 

In [6]:
model.save_pretrained('./fine-tuned-bart')
tokenizer.save_pretrained('./fine-tuned-bart')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine-tuned-bart/tokenizer_config.json',
 './fine-tuned-bart/special_tokens_map.json',
 './fine-tuned-bart/vocab.json',
 './fine-tuned-bart/merges.txt',
 './fine-tuned-bart/added_tokens.json')

In [7]:
import os

model_directory = './fine-tuned-bart'
files = os.listdir(model_directory)
print(files)


['special_tokens_map.json', 'config.json', 'model.safetensors', 'tokenizer_config.json', 'generation_config.json', 'merges.txt', 'vocab.json']


## Inference

In [8]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')
model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')

# Generate a response
def generate_response(question):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=300, num_beams=5, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
question = "I am A.Uha Priya My gender is Female I have done B.Sc My Major subject is Computer Applications I have interest in Cloud computing with skillset of Python, SQL, Java and I don't have certification, with work experience in Linux, Git and I don't have job and my masters degree is Software"
print(generate_response(question))


Uha Priya, with a background in Computer Applications and a keen interest in Cloud Computing, you possess a strong foundation for a career in the tech industry. Your skills in Python, SQL, Java, and certifications in Linux and Git make you well-suited for roles in software development, data analysis, or cloud computing roles. Considering your aspirations, pursuing a career as a Software Developer specializing in cloud computing could be an excellent fit for you. You could leverage your technical skills while leveraging your programming knowledge in cloud technologies. To further enhance your career prospects, consider gaining practical experience through internships or projects to build a strong portfolio. Networking with professionals in the field and staying updated on industry trends will also be beneficial for your career growth. Good luck on your career journey!


In [9]:
df['answer'][0]

"With your educational background in Computer Applications and a Master's degree in Software, along with your strong skills in Python, SQL, Java, Linux, Git, and a certification in Cloud computing, you are well-equipped for a successful career in the tech industry. Given your interest in Cloud computing, you could excel in roles such as Cloud Solutions Architect, Cloud Engineer, or DevOps Engineer where you can leverage your skills in cloud technologies to design, implement, and manage cloud infrastructure for organizations. I recommend continuing to enhance your expertise in Cloud computing by pursuing advanced certifications like AWS Certified Solutions Architect or Microsoft Certified: Azure Solutions Architect Expert to stand out in the competitive tech market. Additionally, gaining experience in real-world cloud projects through internships or freelance opportunities will further boost your profile. Keep networking within the industry and staying updated with the latest technologi

In [12]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')
model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')

# Generate a response
def generate_response(question):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=300, num_beams=5, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
question="name nawaz khan and my education in b-tech mechanical, i have certification in python, google data analysis professional certification, world quant university data science lab, advance business intelligence certification from coursera and certification of deeplearning professional certification from deeplearning.ai assist me for my career"
print(generate_response(question))


Nawaz Khan, with a background in B-Tech mechanical and a keen interest in data science and artificial intelligence, you possess a strong foundation for a successful career in the field of data science. Your certification in Python, Google data analysis professional certification, world quant university data science lab, advanced business intelligence certification from coursera, and certification of deeplearning professional certification from deeplearning.ai further enhance your qualifications for roles in data analysis, machine learning, or artificial intelligence. Considering your skills and interests, pursuing a career as a Data Scientist, Machine Learning Engineer, or AI Specialist could be a rewarding path for you. Networking with professionals in the industry and staying updated on the latest trends and technologies will also be beneficial for your career growth.
